# SQLite3 in Python
In this tutorial, we will use the sqlite3 module in Python to demonstrate how you can write queries and work with a SQLite3 database in Python.  In practice, you will often be working with other types of databases, but the concepts will all be the same so SQLite is a good place to start.

## Setting up the Environment
In order to follow this exercise, you'll need SQLite3 installed.  Binaries for Windows, Linux, and Mac can all be obtained on their [webpage](https://www.sqlite.org/download.html).  Once you have it installed, type `sqlite3` into your OS console.  If you see a promt saying `sqlite>` all is in order and you're ready to proceed.

Now you'll need to get some data.  Head over to the [R Datasets Repository](https://vincentarelbundock.github.io/Rdatasets/datasets.html) and pickup the Titanic and mtcars data set and place them in the same directory as this notebook.

Open up the console in your OS and navigate to the directory containing this notebook.  You'll have to type a command something like `cd [path to directory]`.  Once you are in the proper directory, type `sqlite3` to bring up the SQLite prompt and enter the following.

`.mode csv
.import Titanic.csv Titanic
.import mtcars.csv mtcars`

This imports the data as tables into your SQLite enviroment, named Titanic and mtcars respectively.  In order to use this with Python, we'll have to save it is a database file.  To do so type

`.save myDB.db`

into the SQLite console.  Inside your directory, you should now see a file called `myDB.db` containing the two tables we just imported into SQLite.  Now exit your SQLite session and we're ready to move on to Python.

## sqlite3 Module Basics
Now that we've created a database file, we can start working in Python.  First bring in the `sqlite3` package and execute the following.

In [1]:
import sqlite3

sqlite_db = './myDB.db' # path to db file
conn = sqlite3.connect(sqlite_db) # connect object
c = conn.cursor() # cursor object

The `connect()` function opens a connection to our database and the `cursor` object will be used to issue SQL statements.  Let's try out a simple query on the `Titanic` table.

In [2]:
c.execute('SELECT * FROM Titanic LIMIT 5')
c.fetchall()

[('1', 'Allen, Miss Elisabeth Walton', '1st', '29', 'female', '1', '1'),
 ('2', 'Allison, Miss Helen Loraine', '1st', '2', 'female', '0', '1'),
 ('3', 'Allison, Mr Hudson Joshua Creighton', '1st', '30', 'male', '0', '0'),
 ('4',
  'Allison, Mrs Hudson JC (Bessie Waldo Daniels)',
  '1st',
  '25',
  'female',
  '0',
  '1'),
 ('5', 'Allison, Master Hudson Trevor', '1st', '0.92', 'male', '1', '0')]

And take a look at the mtcars data as well

In [3]:
result = c.execute('SELECT * FROM mtcars LIMIT 5')
for row in result:
    print(row)

('Mazda RX4', '21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4')
('Mazda RX4 Wag', '21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4')
('Datsun 710', '22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1')
('Hornet 4 Drive', '21.4', '6', '258', '110', '3.08', '3.215', '19.44', '1', '0', '3', '1')
('Hornet Sportabout', '18.7', '8', '360', '175', '3.15', '3.44', '17.02', '0', '0', '3', '2')


Note that we used two ways to display the data.  The first `fetchall()` method just returns all returned rows, while the second method formats them a bit more nicely.  It looks like the Titanic dataset contains an index row.  Let's remove it.

In [4]:
c.execute("CREATE TABLE temp AS SELECT Name, PClass, Age, Sex, Survived, SexCode FROM Titanic;")
c.execute("DROP TABLE Titanic")


In [5]:
c.execute("ALTER TABLE temp RENAME TO Titanic;")
conn.commit()
conn.close()

Notice that the last two `conn.commit()` and `conn.close()` calls.  The first commits all changes to the database and the final closes the connection.  It is good practice to always do this.  Let's verify that all worked as intended.

In [6]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("SELECT * FROM Titanic LIMIT 5")
c.fetchall()

[('Allen, Miss Elisabeth Walton', '1st', '29', 'female', '1', '1'),
 ('Allison, Miss Helen Loraine', '1st', '2', 'female', '0', '1'),
 ('Allison, Mr Hudson Joshua Creighton', '1st', '30', 'male', '0', '0'),
 ('Allison, Mrs Hudson JC (Bessie Waldo Daniels)',
  '1st',
  '25',
  'female',
  '0',
  '1'),
 ('Allison, Master Hudson Trevor', '1st', '0.92', 'male', '1', '0')]

In [7]:
conn.close()

And it appears that the unwanted column was sucessfully removed.

**Exercise:** Add a new column, of type `char`, called `class`, to the mtcars data set, containing `low`, `med` or `high` for cars with `mpg` < 15, `mpg` < 20, and `mpg` > 20 respectively.  Be sure to properly open, commit, and close the connection before proceding.

In [8]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("ALTER TABLE mtcars ADD class char;")
c.execute("UPDATE mtcars SET class = CASE WHEN mpg < 15 THEN 'low' \
                                            WHEN mpg < 20 THEN 'med' \
                                            WHEN mpg >= 20 THEN 'high' END;")
conn.commit()
conn.close()

In [9]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("SELECT * FROM mtcars LIMIT 10")
c.fetchall()

[('Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4',
  'high'),
 ('Mazda RX4 Wag',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.875',
  '17.02',
  '0',
  '1',
  '4',
  '4',
  'high'),
 ('Datsun 710',
  '22.8',
  '4',
  '108',
  '93',
  '3.85',
  '2.32',
  '18.61',
  '1',
  '1',
  '4',
  '1',
  'high'),
 ('Hornet 4 Drive',
  '21.4',
  '6',
  '258',
  '110',
  '3.08',
  '3.215',
  '19.44',
  '1',
  '0',
  '3',
  '1',
  'high'),
 ('Hornet Sportabout',
  '18.7',
  '8',
  '360',
  '175',
  '3.15',
  '3.44',
  '17.02',
  '0',
  '0',
  '3',
  '2',
  'med'),
 ('Valiant',
  '18.1',
  '6',
  '225',
  '105',
  '2.76',
  '3.46',
  '20.22',
  '1',
  '0',
  '3',
  '1',
  'med'),
 ('Duster 360',
  '14.3',
  '8',
  '360',
  '245',
  '3.21',
  '3.57',
  '15.84',
  '0',
  '0',
  '3',
  '4',
  'low'),
 ('Merc 240D',
  '24.4',
  '4',
  '146.7',
  '62',
  '3.69',
  '3.19',
  '20',
  '1',
  '0',
  '4',
  '2',
  'high'),
 ('Merc 230',
  '22.8',
  '4',
  

In [10]:
conn.close()

## Security
Very often in practice, you may be working with some web API and will find yourself in a situation where you need to build a query using variables, either provided by the user or otherwise.  One way to do this would be something like the following.

In [12]:
cl = "'high'" # variable to be used in query

conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

results = c.execute("SELECT * FROM mtcars WHERE class = %s" % cl)
for row in results:
    print(row)

('Mazda RX4', '21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4', 'high')
('Mazda RX4 Wag', '21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4', 'high')
('Datsun 710', '22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1', 'high')
('Hornet 4 Drive', '21.4', '6', '258', '110', '3.08', '3.215', '19.44', '1', '0', '3', '1', 'high')
('Merc 240D', '24.4', '4', '146.7', '62', '3.69', '3.19', '20', '1', '0', '4', '2', 'high')
('Merc 230', '22.8', '4', '140.8', '95', '3.92', '3.15', '22.9', '1', '0', '4', '2', 'high')
('Fiat 128', '32.4', '4', '78.7', '66', '4.08', '2.2', '19.47', '1', '1', '4', '1', 'high')
('Honda Civic', '30.4', '4', '75.7', '52', '4.93', '1.615', '18.52', '1', '1', '4', '2', 'high')
('Toyota Corolla', '33.9', '4', '71.1', '65', '4.22', '1.835', '19.9', '1', '1', '4', '1', 'high')
('Toyota Corona', '21.5', '4', '120.1', '97', '3.7', '2.465', '20.01', '1', '0', '3', '1', 'high')
('Fiat X1-9', '27.3', '4', '79', '66', '4.08', 

However, you should **never** write a query in this way.  Reason being, it is very susceptible to [SQL injection attacks](https://en.wikipedia.org/wiki/SQL_injection).  For example, suppose that the variable `cl` was defined by the user via a web interface with a drop down menu, and somehow (in the simplest case by typing the URL) the user set `cl = 'high AND low`.  The result would then be

which is clearly not how we intended the user to access our data.  It isn't too hard to see how an attacker could exploit this to return all sorts of information that we never intended the user to have access to.  Luckily, we can get around this issue pretty easily by doing the following.

In [13]:
cl = ('high', )
results = c.execute("SELECT * FROM mtcars WHERE class = ?", cl)
for row in results:
    print(row)

('Mazda RX4', '21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4', 'high')
('Mazda RX4 Wag', '21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4', 'high')
('Datsun 710', '22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1', 'high')
('Hornet 4 Drive', '21.4', '6', '258', '110', '3.08', '3.215', '19.44', '1', '0', '3', '1', 'high')
('Merc 240D', '24.4', '4', '146.7', '62', '3.69', '3.19', '20', '1', '0', '4', '2', 'high')
('Merc 230', '22.8', '4', '140.8', '95', '3.92', '3.15', '22.9', '1', '0', '4', '2', 'high')
('Fiat 128', '32.4', '4', '78.7', '66', '4.08', '2.2', '19.47', '1', '1', '4', '1', 'high')
('Honda Civic', '30.4', '4', '75.7', '52', '4.93', '1.615', '18.52', '1', '1', '4', '2', 'high')
('Toyota Corolla', '33.9', '4', '71.1', '65', '4.22', '1.835', '19.9', '1', '1', '4', '1', 'high')
('Toyota Corona', '21.5', '4', '120.1', '97', '3.7', '2.465', '20.01', '1', '0', '3', '1', 'high')
('Fiat X1-9', '27.3', '4', '79', '66', '4.08', 

which makes use of sqlite3 module's built in functionality to avoid injection attacks.  In the above, `?` is used as a placeholder for values which will be inserted into the query.  We can also access using the `:` placeholder and a dictionary object.

Which returns the same results but accessed by name.

**Exercise:** What if you wanted to select cars with both `class = 'high'` or `class = 'low` using the `?` place holder like above?  Consult the sqlite3 [documentation](https://docs.python.org/2/library/sqlite3.html) and write a query, using `?`, doing just that and print your results.

In [14]:
cl = ('high', 'low')
results = c.execute("SELECT * FROM mtcars WHERE class IN (?, ?)", cl)
for row in results:
    print(row)

('Mazda RX4', '21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4', 'high')
('Mazda RX4 Wag', '21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4', 'high')
('Datsun 710', '22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1', 'high')
('Hornet 4 Drive', '21.4', '6', '258', '110', '3.08', '3.215', '19.44', '1', '0', '3', '1', 'high')
('Duster 360', '14.3', '8', '360', '245', '3.21', '3.57', '15.84', '0', '0', '3', '4', 'low')
('Merc 240D', '24.4', '4', '146.7', '62', '3.69', '3.19', '20', '1', '0', '4', '2', 'high')
('Merc 230', '22.8', '4', '140.8', '95', '3.92', '3.15', '22.9', '1', '0', '4', '2', 'high')
('Cadillac Fleetwood', '10.4', '8', '472', '205', '2.93', '5.25', '17.98', '0', '0', '3', '4', 'low')
('Lincoln Continental', '10.4', '8', '460', '215', '3', '5.424', '17.82', '0', '0', '3', '4', 'low')
('Chrysler Imperial', '14.7', '8', '440', '230', '3.23', '5.345', '17.42', '0', '0', '3', '4', 'low')
('Fiat 128', '32.4', '4', '78.7', '

In [15]:
conn.close()

## Inserting Data and Error Checking
You may find yourself in a situation where you want to insert new data into an existing table in your database.  This isn't too hard to do with the sqlite3 module, but since you will be committing permanent changes to the database, you will want to do so with caution, performing proper error checking as you move along.

Before getting into inserts and error checks, let's make a copy of the `Titanic` table that we can play with.

In [16]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("CREATE TABLE Titanic_temp AS SELECT * FROM Titanic;")

conn.commit()
conn.close()

And make sure everything appears as it should.

In [18]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("SELECT * FROM Titanic_temp LIMIT 10")
c.fetchall()

[('Allen, Miss Elisabeth Walton', '1st', '29', 'female', '1', '1'),
 ('Allison, Miss Helen Loraine', '1st', '2', 'female', '0', '1'),
 ('Allison, Mr Hudson Joshua Creighton', '1st', '30', 'male', '0', '0'),
 ('Allison, Mrs Hudson JC (Bessie Waldo Daniels)',
  '1st',
  '25',
  'female',
  '0',
  '1'),
 ('Allison, Master Hudson Trevor', '1st', '0.92', 'male', '1', '0'),
 ('Anderson, Mr Harry', '1st', '47', 'male', '1', '0'),
 ('Andrews, Miss Kornelia Theodosia', '1st', '63', 'female', '1', '1'),
 ('Andrews, Mr Thomas, jr', '1st', '39', 'male', '0', '0'),
 ('Appleton, Mrs Edward Dale (Charlotte Lamson)',
  '1st',
  '58',
  'female',
  '1',
  '1'),
 ('Artagaveytia, Mr Ramon', '1st', '71', 'male', '0', '0')]

In [19]:
conn.close()

All looks in order.  Now let's try inserting some data into the newly created `Titanic_temp` table.

In [20]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("INSERT INTO Titanic_temp VALUES ('Jane', '2nd', '29', 'female', '1')")
results = c.execute("SELECT * FROM Titanic_temp LIMIT 5")
for row in results:
    print(row)
    
conn.close()

OperationalError: table Titanic_temp has 6 columns but 5 values were supplied

Notice that we got back an error indicating that we didn't provide enough data points for a complete row.  It's good practice to have a way for our code to handle this type of error.  So instead, let's do

In [21]:
import sys

conn = None

try:
    conn = sqlite3.connect(sqlite_db)
    c = conn.cursor()

    c.execute("INSERT INTO Titanic_temp VALUES ('Jane', '2nd', '29', 'female', '1')")
    results = c.execute("SELECT * FROM Titanic_temp LIMIT 5")
    for row in results:
        print(row)
        
except sqlite3.Error as e:
    print("Error %s:" % e.args[0])
    
finally:
    if conn:
        conn.close()

Error table Titanic_temp has 6 columns but 5 values were supplied:


The error is now caught by the interpreter, preventing our data from becoming corrupted or halting the program (if that is what we intended).  

The above works by first setting the `conn` vairable to `None` so that if there is an error connecting to the database, the error can be handled in the `finally` clause.  The `try` statements, tells the interpreter to "try" this block of code, and if there is an exception, the `exception` clause tells it how to handle the error.

There is however, a much more concise way to do this using `with`.

In [22]:
conn = sqlite3.connect(sqlite_db)

with conn:
    conn = sqlite3.connect(sqlite_db)
    c = conn.cursor()

    c.execute("INSERT INTO Titanic_temp VALUES ('Jane', '2nd', '29', 'female', '1')")
    results = c.execute("SELECT * FROM Titanic_temp LIMIT 5")
    for row in results:
        print(row)

OperationalError: table Titanic_temp has 6 columns but 5 values were supplied

We don't get the custom error handling, but by using `with`, the resource `conn` is closed automatically upon sucessful completion or an error.  Situations where we want to handle errors in specific ways will lend themselves to a `try` statement, but for most of our puposes, the more concice `with` works better.  Let's fix the query so that it should insert itself correctly into the database now.

In [23]:
conn = sqlite3.connect(sqlite_db)

with conn:
    conn = sqlite3.connect(sqlite_db)
    c = conn.cursor()

    c.execute("INSERT INTO Titanic_temp VALUES ('Jane', '2nd', '29', 'female', '1', '0')")
    results = c.execute("SELECT * FROM Titanic_temp WHERE name = 'Jane'")
    for row in results:
        print(row)
        
    conn.close()

('Jane', '2nd', '29', 'female', '1', '0')


Typically, we'll want to add more than just one row to our table.  To add many rows, we can use the `executemany()` function.

In [24]:
conn = sqlite3.connect(sqlite_db)

new_names = ('John', 'Tim', 'Anna', 'Rose', 'Jack')
new_data = [('John', '2nd', '35', 'male', '1', '0'),
           ('Tim', '2nd', '5', 'male', '1', '0'),
           ('Anna', '3rd', '24', 'female', '1', '1'),
           ('Rose', '1st', '17', 'female', '1', '1'),
           ('Jack', '3rd', '21', 'male', '0', '0')]

with conn:
    conn = sqlite3.connect(sqlite_db)
    c = conn.cursor()
    
    c.executemany('INSERT INTO Titanic_temp VALUES (?, ?, ?, ?, ?, ?)', new_data)
    results = c.execute("SELECT * FROM Titanic_temp WHERE Name IN (?, ?, ?, ?, ?)", new_names)
    for row in results:
        print(row)
    
    conn.commit()

('John', '2nd', '35', 'male', '1', '0')
('Tim', '2nd', '5', 'male', '1', '0')
('Anna', '3rd', '24', 'female', '1', '1')
('Rose', '1st', '17', 'female', '1', '1')
('Jack', '3rd', '21', 'male', '0', '0')


We can also access the data using the row names by making use of the `Row`, and `row_factory` attributes.

In [25]:
conn = sqlite3.connect(sqlite_db)

with conn:
    
    conn.row_factory = sqlite3.Row
    
    cur = conn.cursor()
    cur.execute("SELECT * FROM Titanic LIMIT 5")
    
    rows = cur.fetchall()
    
    for row in rows:
        print("%s %s %s" % (row['Name'], row['PClass'], row['Age']))
    

Allen, Miss Elisabeth Walton 1st 29
Allison, Miss Helen Loraine 1st 2
Allison, Mr Hudson Joshua Creighton 1st 30
Allison, Mrs Hudson JC (Bessie Waldo Daniels) 1st 25
Allison, Master Hudson Trevor 1st 0.92


Allows accesing the columns by name.

When working with databases, you may find that you want to access the metadata, or, information about the data contained within the database.  To do this in sqlite3, run the following.

In [26]:
conn = sqlite3.connect(sqlite_db)

with conn:
    cur = conn.cursor()
    
    cur.execute("PRAGMA table_info(Titanic)")
    data = cur.fetchall()
    
    for d in data:
        print(d[0], d[1], d[2])

0 Name TEXT
1 PClass TEXT
2 Age TEXT
3 Sex TEXT
4 Survived TEXT
5 SexCode TEXT


Which returns information regarding column names and datatypes.

**Exercise:** Like we did above, create a copy of the `mtcars` table, and insert 3 new cars into the table, using only a subset of the columns, with proper error checking using a `try` statement, verifying that they were properly inserted before commiting your changes.

In [27]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("CREATE TABLE mtcars_temp AS SELECT * FROM mtcars")

conn.commit()
conn.close()

In [28]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("SELECT * FROM mtcars_temp LIMIT 10")
c.fetchall()


[('Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4',
  'high'),
 ('Mazda RX4 Wag',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.875',
  '17.02',
  '0',
  '1',
  '4',
  '4',
  'high'),
 ('Datsun 710',
  '22.8',
  '4',
  '108',
  '93',
  '3.85',
  '2.32',
  '18.61',
  '1',
  '1',
  '4',
  '1',
  'high'),
 ('Hornet 4 Drive',
  '21.4',
  '6',
  '258',
  '110',
  '3.08',
  '3.215',
  '19.44',
  '1',
  '0',
  '3',
  '1',
  'high'),
 ('Hornet Sportabout',
  '18.7',
  '8',
  '360',
  '175',
  '3.15',
  '3.44',
  '17.02',
  '0',
  '0',
  '3',
  '2',
  'med'),
 ('Valiant',
  '18.1',
  '6',
  '225',
  '105',
  '2.76',
  '3.46',
  '20.22',
  '1',
  '0',
  '3',
  '1',
  'med'),
 ('Duster 360',
  '14.3',
  '8',
  '360',
  '245',
  '3.21',
  '3.57',
  '15.84',
  '0',
  '0',
  '3',
  '4',
  'low'),
 ('Merc 240D',
  '24.4',
  '4',
  '146.7',
  '62',
  '3.69',
  '3.19',
  '20',
  '1',
  '0',
  '4',
  '2',
  'high'),
 ('Merc 230',
  '22.8',
  '4',
  

In [29]:
conn.close()

In [34]:
conn = None

new_cars = ('Land Rover', 'Maserati', 'Rolls Royce')
new_data = [('Land Rover', '17', '6', '150', '130', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'med'),
           ('Maserati', '19', '8', '190', '170', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'med'),
           ('Rolls Royce', '15', '8', '160', '120', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'low')]

try:
    conn = sqlite3.connect(sqlite_db)
    c = conn.cursor()
    
    c.executemany("INSERT INTO mtcars_temp VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", new_data)
    results = c.execute("SELECT * FROM mtcars_temp WHERE name IN (?, ?, ?)", new_cars)
    for row in results:
        print(row)
except sqlite3.Error as e:
    print("Error %s:" % e.args[0])
    
finally:
    if conn:
        conn.commit()
        conn.close()

('Land Rover', '17', '6', '150', '130', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'med')
('Maserati', '19', '8', '190', '170', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'med')
('Rolls Royce', '15', '8', '160', '120', '4.2', '2.63', '17.03', '0', '1', '4', '4', 'low')


In [35]:
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

c.execute("SELECT * FROM mtcars_temp WHERE name = 'Maserati'")
c.fetchall()


[('Maserati',
  '19',
  '8',
  '190',
  '170',
  '4.2',
  '2.63',
  '17.03',
  '0',
  '1',
  '4',
  '4',
  'med')]

In [36]:
conn.close()

## Pandas
Now that we have seen how the sqlite3 module functions, we can work with the data more directly in our Python enviroment by importing it into a Pandas dataframe, providing you with all the tools that you are used to using.

Doing this in Pandas is very easy.  All we have to do is open a connection with our database using sqlite3, and then make use of the Pandas `read_sql_query()` function.

In [37]:
import sqlite3
import pandas as pd

sqlite_db = './myDB.db'
conn = sqlite3.connect(sqlite_db)

titanic_data = pd.read_sql_query("select * from Titanic", conn)
titanic_data.head()

,Name,PClass,Age,Sex,Survived,SexCode
0,"Allen, Miss Elisabeth Walton",1st,29,female,1,1
1,"Allison, Miss Helen Loraine",1st,2,female,0,1
2,"Allison, Mr Hudson Joshua Creighton",1st,30,male,0,0
3,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25,female,0,1
4,"Allison, Master Hudson Trevor",1st,0.92,male,1,0


We can also pass whatever options we'd like to `read_sql_query()` as well.  For example

In [38]:
titanic_survivors = pd.read_sql_query("select * from Titanic where Survived = 1", conn)
titanic_survivors.head()

,Name,PClass,Age,Sex,Survived,SexCode
0,"Allen, Miss Elisabeth Walton",1st,29,female,1,1
1,"Allison, Master Hudson Trevor",1st,0.92,male,1,0
2,"Anderson, Mr Harry",1st,47,male,1,0
3,"Andrews, Miss Kornelia Theodosia",1st,63,female,1,1
4,"Appleton, Mrs Edward Dale (Charlotte Lamson)",1st,58,female,1,1


We can also create new tables in our database using Pandas.  Let's say we want to add the `titanic_survivors` dataframe to our database permanently.  We can do so by calling

In [39]:
titanic_survivors.to_sql("Titanic_survivors", conn)

And then query the results to make sure it worked.

In [40]:
pd.read_sql_query("select * from Titanic_survivors", conn).head()

,index,Name,PClass,Age,Sex,Survived,SexCode
0,0,"Allen, Miss Elisabeth Walton",1st,29,female,1,1
1,1,"Allison, Master Hudson Trevor",1st,0.92,male,1,0
2,2,"Anderson, Mr Harry",1st,47,male,1,0
3,3,"Andrews, Miss Kornelia Theodosia",1st,63,female,1,1
4,4,"Appleton, Mrs Edward Dale (Charlotte Lamson)",1st,58,female,1,1


In [41]:
conn.close()

Note that the `to_sql()` function takes a parameter called `if_exists` which specifies what action to take when writing to the database.  If we set this parameter equal to `replace`, then it will replace any table with the same name, altering the table.

**Exercise:** Create a Pandas dataframe from the mtcars dataset, containing only cars with 6 cylinder engines and then create a new table in the database from this dataframe.  Verify that all operations worked.

In [42]:
conn = sqlite3.connect(sqlite_db)

mtcars_df = pd.read_sql_query("select * from mtcars", conn)
mtcars_df.head()

,name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,class
0,Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4,high
1,Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4,high
2,Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1,high
3,Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,high
4,Hornet Sportabout,18.7,8,360,175,3.15,3.44,17.02,0,0,3,2,med


In [43]:
cyl6_df = pd.read_sql_query("select * from mtcars where cyl = 6", conn)
cyl6_df.head()

,name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,class
0,Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4,high
1,Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4,high
2,Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,high
3,Valiant,18.1,6,225,105,2.76,3.46,20.22,1,0,3,1,med
4,Merc 280,19.2,6,167.6,123,3.92,3.44,18.3,1,0,4,4,med


In [44]:
cyl6_df.to_sql("mtcars_6cyl", conn)

In [45]:
pd.read_sql_query("select * from mtcars_6cyl", conn).head()

,index,name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,class
0,0,Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4,high
1,1,Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4,high
2,2,Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,high
3,3,Valiant,18.1,6,225,105,2.76,3.46,20.22,1,0,3,1,med
4,4,Merc 280,19.2,6,167.6,123,3.92,3.44,18.3,1,0,4,4,med
